## HuggingFace로 두 문장의 논리적 모순 분류하기

이번 과제는 자연어 task 중 하나인 MNLI를 해결하는 모델을 HuggingFace로 학습하는 것입니다. MNLI를 요약하면 다음과 같습니다.

- **입력**: premise에 해당하는 문장과 hypothesis에 해당하는 문장 두 개가 입력으로 들어옵니다.
- **출력:** 분류 문제로, 두 문장이 들어왔을 때 다음 세 가지를 예측하시면 됩니다.
    - **Entailment:** 두 문장에 논리적 모순이 없습니다.
    - **Neutral:** 두 문장은 논리적으로 관련이 없습니다.
    - **Contradiction:** 두 문장 사이에 논리적 모순이 존재합니다.

이 때, 다음 요구사항이 담긴 colab notebook을 만들어내시면 됩니다:

- `load_dataset("nyu-mll/glue", "mnli")` 로 dataset을 불러옵니다.
    - 학습 때는 `train` split만 활용하셔야 합니다. 나머지 split은 사용불가입니다.
    - Validation data가 필요한 경우, `train` split에서 가져오셔야 합니다.
- `trainer.train()`를 통해 학습된 log가 남아있어야 합니다.
- Dataset의 `validation_matched`에 대한 성능을 출력하고, 50%를 넘기셔야 합니다.

이전 과제와 똑같이 validation data 유무, 모델 architecture, hyper-parameter 등은 위의 조건만 만족한다는 가정 하에서 마음대로 수정하셔도 됩니다.

필수 패키지 설치

In [1]:
%pip install transformers datasets evaluate

Note: you may need to restart the kernel to use updated packages.


패키지 로딩

In [3]:
import evaluate

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification

MNIL 데이터셋 로딩

In [5]:
statements = load_dataset("nyu-mll/glue", "mnli")
statements

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 392702
    })
    validation_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9815
    })
    validation_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9832
    })
    test_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9796
    })
    test_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9847
    })
})

학습 데이터셋 확인 


In [11]:
statements['train'][0]

{'premise': 'Conceptually cream skimming has two basic dimensions - product and geography.',
 'hypothesis': 'Product and geography are what make cream skimming work. ',
 'label': 1,
 'idx': 0}

BERT 토큰나이저 사용
학습 데이터셋에서 Premise와 Hypothesis를 합처서 하나의 문장으로 구성함

In [14]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def preprocess_function(data):
    return tokenizer(f'{data["premise"]}, {data["hypothesis"]}', truncation=True)

statement_tokenized = statements.map(preprocess_function, batch_size=True)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/392702 [00:00<?, ? examples/s]

Map:   0%|          | 0/9815 [00:00<?, ? examples/s]

Map:   0%|          | 0/9832 [00:00<?, ? examples/s]

Map:   0%|          | 0/9796 [00:00<?, ? examples/s]

Map:   0%|          | 0/9847 [00:00<?, ? examples/s]

토큰화된 학습 데이터셋 확인

In [19]:
statement_tokenized['train'][0]

{'premise': 'Conceptually cream skimming has two basic dimensions - product and geography.',
 'hypothesis': 'Product and geography are what make cream skimming work. ',
 'label': 1,
 'idx': 0,
 'input_ids': [101,
  28103,
  14795,
  7081,
  10458,
  25004,
  1144,
  1160,
  3501,
  10082,
  118,
  3317,
  1105,
  14534,
  119,
  117,
  22249,
  1105,
  14534,
  1132,
  1184,
  1294,
  7081,
  10458,
  25004,
  1250,
  119,
  102],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

Validation 셋을 학습 데이터셋에서 분리하여 만듦(20%)

In [21]:
train_split = statement_tokenized['train'].train_test_split(test_size=0.2)
trainset, validationset = train_split['train'], train_split['test']
testset = statement_tokenized['validation_matched']

In [22]:
len(trainset), len(validationset), len(testset)

(314161, 78541, 9815)

Pre-train된 BERT 모델을 세팅함

In [24]:
from transformers import BertConfig

config = BertConfig()

config.hidden_size = 128
config.intermediate_size = 128
config.num_hidden_layers = 4
config.num_attention_heads = 4
config.num_labels = 3


model = AutoModelForSequenceClassification.from_config(config)

학습 횟수와 learning rate 등 중요 하이퍼 파라미터 세팅

In [28]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='./hf_transformer',  # 모델, log 등을 저장할 directory
    num_train_epochs=10,  # epoch 수
    per_device_train_batch_size=128,  # training data의 batch size
    per_device_eval_batch_size=128,  # validation data의 batch size
    logging_strategy="epoch",  # Epoch가 끝날 때마다 training loss 등을 log하라는 의미
    do_train=True,  # 학습을 진행하겠다는 의미
    do_eval=True,  # 학습 중간에 validation data에 대한 평가를 수행하겠다는 의미
    eval_strategy="epoch",  # 매 epoch가 끝날 때마다 validation data에 대한 평가를 수행한다는 의미
    save_strategy="epoch",  # 매 epoch가 끝날 때마다 모델을 저장하겠다는 의미
    learning_rate=1e-3,  # optimizer에 사용할 learning rate
    load_best_model_at_end=True  # 학습이 끝난 후, validation data에 대한 성능이 가장 좋은 모델을 채택하겠다는 의미
)

Validation 데이터셋을 이용해 학습 중 모델 정확도 측정 함수 구현

In [29]:
import numpy as np

accuracy = evaluate.load("accuracy")

def compute_metrics(pred):
    predictions, labels = pred
    predictions = np.argmax(predictions, axis=1)
    
    return accuracy.compute(predictions=predictions, references=labels)

학습에 필요한 함수 및 세팅값으로 Trainer instance 만듦

In [30]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=trainset,
    eval_dataset=validationset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

학습 진행

In [31]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.027200,1.004139,0.490126
2,0.982400,0.974588,0.513592
3,0.951100,0.953793,0.532614
4,0.920100,0.951530,0.538878
5,0.888000,0.952482,0.540278
6,0.855200,0.963506,0.542787
7,0.821900,0.981828,0.539756
8,0.789100,1.015478,0.534689
9,0.756600,1.041254,0.528934
10,0.729000,1.081714,0.526171


TrainOutput(global_step=24550, training_loss=0.8720520442177954, metrics={'train_runtime': 3087.5323, 'train_samples_per_second': 1017.515, 'train_steps_per_second': 7.951, 'total_flos': 998349812888886.0, 'train_loss': 0.8720520442177954, 'epoch': 10.0})

테스트 데이터셋으로 모델의 정확도 측정

In [32]:
trainer.evaluate(testset)

{'eval_loss': 0.940912663936615,
 'eval_accuracy': 0.5464085583290881,
 'eval_runtime': 6.0992,
 'eval_samples_per_second': 1609.227,
 'eval_steps_per_second': 12.625,
 'epoch': 10.0}

학습된 모델 파일로 저장

In [33]:
trainer.save_model()